# <center> Домашнее задание 3 </center>

## 1. Матрицы.

Задачи этого раздела нужно сдать в контесте.

**Тем не менее, финальное решение нужно вставить в этот ноутбук**



### 1.1 Нетеплицева матрица (15 баллов)
Напишите функцию, которая на вход принимает вектор из целых чисел, а выдает [матрицу Тёплица](https://en.wikipedia.org/wiki/Toeplitz_matrix), но не совсем обычную: каждая вторая строка должна быть развернута в обратном порядке относительно настоящей матрицы Тёплица. В решении нельзя использовать готовые реализации из ```scipy.linalg``` или других библиотек, но необходимо использовать ```numpy```. 

In [ ]:
import numpy as np

def Toeplitz(vec):
  npVec = np.array(vec, "int")
  npMatrix = np.empty((npVec.shape[0], npVec.shape[0]), "int")
  
  for i in range(npVec.shape[0]):
    for j in range(npVec.shape[0]):
      if not i+j >= npMatrix.shape[0]:
        npMatrix[i+j, i] = npVec[j]
        npMatrix[i, i+j] = npVec[j]

  return npMatrix

def performMatrixTransformation(mat):
  for i in range(mat.shape[0]):
    if not i % 2 == 0:
      for j in range(int(mat.shape[1]/2)):
        mat[i, mat.shape[1]-1-j], mat[i, j] = mat[i, j], mat[i, mat.shape[1]-1-j]
  return mat


vec = [int(i) for i in input().strip().split(" ")]
print(performMatrixTransformation(Toeplitz(vec)))

### 1.2 Подматрица с нужным детерминантом (15 баллов)
У вас есть две матрицы X и Y одинакового размера. Теперь возьмем от марицы X некоторую подматрицу M и назовем ее "координатами" координату ее верхнего левого элемента в матрице X (то есть, координату левого верхнего угла матрицы M).

Проверьте, есть ли в матрице X такая подматрица M, что det(M) = Y[i, j], где i, j это "координаты" матрицы M.

In [ ]:
import numpy as np

def inputMatrices():
  inpX = input().strip()
  inpY = input().strip()

  if not ";" in inpX:
    return np.matrix([[int(inpX)]], "int"), np.matrix([[int(inpY)]], "int")
  
  x = []
  for i, row in enumerate(inpX.split(";")):
    x.append([])
    for col in row.split(" "):
      x[i].append(col)
  
  y = []
  for i, row in enumerate(inpY.split(";")):
    y.append([])
    for col in row.split(" "):
      y[i].append(col)
  
  return np.matrix(x, "int"), np.matrix(y, "int")

def findMMats(X, i, j):
  last = min(X.shape[0]-i, X.shape[1]-j)
  mats = []
  for ii in range(1, last+1):
    mats.append(X[i:i+ii, j:j+ii])

  return mats

def solve(X,Y):
  for i in range(Y.shape[0]):
    for j in range(Y.shape[1]):
      Ms = findMMats(X,i,j)
      dets = [round(np.linalg.det(x)) for x in Ms]
      if Y[i,j] in dets:
        print("True")
        return
  
  print('False')

X,Y = inputMatrices()
solve(X,Y)

In [ ]:
Y.shape

## 2. Пандас Фигандас

Задачи этого раздела нужно сдать в контесте.

**Тем не менее, финальное решение нужно вставить в этот ноутбук и показать явно финальные таблички**

Менеджер Пандас Фигандас работает в одной испанкой компании. Он прошел часовой интенсив по питону, где опытные аналитики с многолетним стажем научили его самой главной библиотеке на питоне - pandas! Используя лишь пару строк кода, Пандас Фигандас научился виртуозно считать средние и дисперсии. Можно сказать, что он стал повелителем первичного анализа данных!

Тем не менее, он нанял вас решения для некоторых "нестандартных" задач. Взамен он обещал надавить на семинаристов курса по питону, чтобы вам повысили оценки за ДЗ-3.

In [ ]:
import pandas as pd

### 2.1. Коммиты (20 баллов)

По результатам полугодия команда Пандаса Фигандаса не смогла выполнить kpi, который он им поставил. Он задумался, а нет ли в его команде бездельников? Он смог найти файл (```commits.json```) с последними событиями в репозитории и собирается посчитать научно обоснованный показатель эффективности разработчика - количество написанного кода (в строках), количество добавленных файлов и количество комитов. Но вот же беда - данные ведь в формате json! Помогите ему и предоставьте pd.DataFrame таблицу! Таблица должна содержать колонки в следующем порядке ```["username", "commits", "changed_lines", "new_files"]```, а также должна быть отсортирована по username. 

* username - юзернейм разработчика
* commits - количество комитов разработчика
* changed_lines - суммарное количество changed_lines
* new_files - суммарное количество новых файлов добавленных разработчиком. Новым файлом считается любой файл, которого вы раньше не встречали (если смотреть на commit_time). Обратите внимание, что данные изначально не отсортированы по commit_time



In [ ]:
import json
from datetime import datetime
import pandas as pd 

commits = []

with open('input.txt') as json_file:
    data = json.load(json_file)
    for commit in data:
      commitData = {}
      commitData["username"] = commit["username"]
      commitData["time"] = datetime.strptime(commit["commit_time"], "%Y/%m/%d %H:%M:%S")
      
      files = []
      for f in commit["files"]:
        fileData = {}
        fileData["filename"] = f["name"]
        fileData["lines"] = int(f["changed_lines"])
        files.append(fileData)
      
      commitData["files"] = files 
      commits.append(commitData)

def getUserCommits(username):
  return [x for x in commits if x["username"] == username]

def getUserChangedFiles(username):
  files = []
  for c in getUserCommits(username):
    for f in c["files"]:
      files.append((c["time"], f))
  return files

def getUserChangedLines(username):
  lines = 0
  for f in getUserChangedFiles(username):
    lines += f[1]["lines"] 
  return lines

def getUsersFiles():
  usersFiles = []
  for u in users:
    for f in getUserChangedFiles(u):
      usersFiles.append((u, f[0], f[1]["filename"]))
  
  return usersFiles

def getFileNames():
  return list(set([f[2] for f in getUsersFiles()]))

def getFileCreators():
  uFiles = sorted(getUsersFiles(), key=lambda uf:uf[1])
  creators = []
  for f in getFileNames():
    ff = [file for file in uFiles if f == file[2]][0]
    creators.append((ff[0], ff[2]))
  
  return creators

def getFilesCreatedByUser(username):
  return [f for f in getFileCreators() if f[0] == username]

users = list(set([x["username"] for x in commits]))
out = [(u, len(getUserCommits(u)), getUserChangedLines(u), len(getFilesCreatedByUser(u))) for u in users]

df = pd.DataFrame.from_records(out, columns=["username", "commits", "changed_lines", "new_files"])
print(df.sort_values(by=["username"], ascending=True).to_string(index=False))

### 2.2. Нетворкинг (10 баллов)

Связи - очень важная составляющая успеха любого современного менеджера. Пандас Фигандас хочет понять, кто из его знакомых имеет больше всего связей. Для этого он выписал в файлик (```networking.tsv```) пары имен, означающие дружеские связи между ними. Посчитайте для каждого человека, сколько знакомых он имеет, отсортируйте по убыванию и предоставьте Пандасу Фигандосу pd.DataFrame табличку.

Таблица должна содержать колонки в следующем порядке ```["person", "links_count"]```, а также должна быть отсортирована (по убыванию) по links_count и содержать только 5 строк с самым большим links_count. 

* person - имя человека
* links_count - количество знакомых ему людей

In [ ]:
import pandas as pd

dfin = pd.read_csv("data/networking.tsv", sep="\t", names=["personA", "personB"]).drop_duplicates()
dfout = pd.DataFrame(df["personA"].append(df["personB"]).reset_index(drop=True).unique(), columns=["person"])
dfout['links_count'] = dfout.person.map(lambda p: dfin[(dfin.personA == p) | (dfin.personB == p)].shape[0])
dfout = dfout.sort_values(by=["links_count"], ascending=False).reset_index(drop=True)

print(dfout.head().to_string(index=False))

pandas почему-то не работает в контесте. переделаем на другой вариант.


In [ ]:
import pandas as pd

links = {}
with open('input.txt', 'r') as tsv:
  for line in tsv:
    personA, personB = line.strip().split("\t")
    if (links.get(personA, None) == None):
      links[personA] = 1
    else:
      links[personA] += 1
    
    if (links.get(personB, None) == None):
      links[personB] = 1
    else:
      links[personB] += 1
    
links = sorted(list(links.items()), key=lambda x: (-x[1], x[0]))

df = pd.DataFrame.from_records(links, columns=["person", "links_count"])
print(df.head().to_string(index=False))


## 3. Дефолты (40 баллов)

Судя по стоимости недвижимости в Москве, придется влезть в долги... Проанализируем, как часто люди не отдают ипотеки и кредиты! 

В файле ```defaults.csv``` предоставлены данные, которые можно использовать для анализа клиентской базы, предсказания дефолта, кредитного скоринга (т.е. для оценки кредитных рисков клиентов). 

Переменные:
* **ID** - идентификатор клиента;
* **LIMIT_BAL** - сумма предоставленного кредита/ипотеки (в долларах);
* **SEX** - пол (1=мужской, 2=женский);
* **EDUCATION** - уровень образования (1=научная степень, 2=высшее, 3=среднее, 4=другое, 5 и 6=неизвестно);
* **MARRIAGE** - семейное положение (1=женат/замужем, 2=холост/незамужняя, 3=другое);
* **AGE** - возраст (в годах);
* **PAY_0** - Статус погашения кредита в сентябре (-1 = надлежащая оплата, 1 = задержка платежа на один месяц, 2 = задержка платежа на два месяца, ... 8 = задержка платежа на восемь месяцев, 9 = задержка платежа на девять месяцев и более). **PAY_2, PAY_3, ..., PAY_6** - в августе, июле, июне, мае и апреле соответственно;
* **BILL_AMT1** - Сумма выписки по счету в сентябре (в долларах). **BILL_AMT2, BILL_AMT3, ..., BILL_AMT6** - в августе, июле, июне, мае и апреле соответственно;
* **PAY_AMT1** - Сумма предыдущего платежа в сентябре (в долларах). **PAY_AMT1, PAY_AMT2, ..., PAY_AMT6** - в августе, июле, июне, мае и апреле соответственно;
* **default.payment.next.month**: произошел дефолт (1=да, 0=нет).

Нужно проанализировать набор данных банка, выполнив следующие задания:
1. Загрузить данные и преобразуйте в pd.DataFrame. Какого размера этот датасет? Покажите первые 10 строк ваших данных. Покажите 10 рандомных строк ваших данных.<br>
2. Отобразите количество пропущенных значений во всех колонках.  <br>
3. Выведите количество уникальных клиентов в датасете. <br>
4. Проверьте, есть ли среди клиентов дисбаланс в количестве женщин и мужчин? Какого их соотношение (в процентах)? <br>
5. Постройте график распределения клиентов по значениям переменной *AGE*. Клиенты каких возрастов чаще всего приходят за кредитом в этот банк?<br>
6. Выведите общее количество клиентов с дефолтом и без него. Постройте гистограмму с процентным отношением этих групп. Как можно оценить, велика ли полученная доля клиентов с дефолтом? <br>
7. Постройте гистограмму по суммам предоставленных кредитов (*LIMIT_BAL*) для каждой из групп клиентов отдельно (с дефолтом и без). (один график)
8. На основе графика из предыдущего пункта, ответьте на вопросы: <br>
8.1 По кредитам какого размера чаще всего происходят дефолты? <br>
8.2 Начиная от какой суммы кредита клиентов без дефолта становится больше, чем клиентов с дефолтом? <br>
8.3  Как по графику определить, есть ли такие одобренные суммы кредитов, по которым никогда не происходит дефолт? Выведите значения а) сумм кредитов, по которым всегда встречается дефолт, б) и те, по которым никогда не бывает дефолта. <br>
9. Постройте матрицу корреляций для всех переменных по статусу погашения кредита. Добавьте значения корреляций на график. Есть ли общий тренд по возрастанию/убыванию корреляции по времени (всем месяцам)? <br>
10. Напишите функцию, которая возвращает словарь, где ключ - это сумма одобренного кредита, а значение - это вложенный словарь. В нем по ключу ```ratio``` хранится отношение клиентов с дефолтом к тем, у кого его не было, с такой суммой кредитов, а по ключу ```num_clients```  - общее число клиентов с таким кредитом. Пример элемента словаря: ``` {50000.0 : {'ratio': 0.357, 'num_clients': 3365}```. Отсортируйте получившийся словарь по ключу ```ratio``` в убывающем порядке. У какого размера кредита наибольшая доля клиентов с дефолтом? А сколько в датасете клиентов, по которым посчитана эта доля? Можем ли мы сделать вывод, что в целом не стоит одобрять эту сумму для кредита новым клиентам? <br>

Важно! Каждый график должен содержать как минимум заголовок и подписанные оси. 

**Задание 1**

In [ ]:
import pandas as pd
df = pd.read_csv("default.csv")

In [ ]:
df.shape

30000 строк, 25 столбцов

In [ ]:
df.head(10)

In [ ]:
df.sample(10)

**Задание 2**

In [ ]:
df.isna().replace({True:1, False:0}).sum(axis=0)

**Задание 3**

In [ ]:
len(df["ID"].unique())

**Задание 4**

In [ ]:
sex = pd.DataFrame(df.SEX.value_counts()).reset_index()
sex["index"].replace({1:"male", 2:"female"}, inplace=True)
sex = sex.rename(columns={"index":"sex", "SEX":"count"})
sex["percent"] = sex["count"]/sex["count"].sum()
sex

Дизбаланс есть. Женщин = 60%, мужчин = 40%

**Задание 5**

In [ ]:
df.AGE.hist()

Чаще всего возраст 28-38...

**Задание 6**

In [ ]:
df["default.payment.next.month"].sum()

In [ ]:
df["default.payment.next.month"].hist()

In [ ]:
df["default.payment.next.month"].sum()/df["default.payment.next.month"].count()

In [ ]:
(df["default.payment.next.month"].count()-df["default.payment.next.month"].sum())/df["default.payment.next.month"].count()

Видим, что дефолтных 6636, недефолтных 23364.
Процентное соотношение ~ 22%/78%
Согласно принципу Паретто, соотношение нормальное. 

**Задание 7. Постройте гистограмму по суммам предоставленных кредитов (LIMIT_BAL) для каждой из групп клиентов отдельно (с дефолтом и без). (один график)**

In [ ]:
df["LIMIT_BAL"].hist(bins=50,by=df["default.payment.next.month"])

На основе графика из предыдущего пункта, ответьте на вопросы:
1 По кредитам какого размера чаще всего происходят дефолты?
Чаще всего (см. график 2, первый столбец) дефолты происходят по небольшим кредитам.  
2 Начиная от какой суммы кредита клиентов без дефолта становится больше, чем клиентов с дефолтом?  
Подобное свойство наблюдается в районе 0. К сожалению, из-за масштаба графика сказать точно не получится. Но практически сразу же число недефолтных клиентов превалирует над дефолтными.  
3 Как по графику определить, есть ли такие одобренные суммы кредитов, по которым никогда не происходит дефолт? 
Это суммы больше 450-500k.  


Выведите значения  
а) сумм кредитов, по которым всегда встречается дефолт,  
б) и те, по которым никогда не бывает дефолта.

In [ ]:
default = set(df["LIMIT_BAL"].loc[df["default.payment.next.month"] == 0].unique())
without_default = set(df["LIMIT_BAL"].loc[df["default.payment.next.month"] == 1].unique())

a)

In [ ]:
without_default - default

b)

In [ ]:
default - without_default

**Задание 12.**

In [ ]:
corrs = []
for column in df.columns:
  corrs.append(df[column].corr(df["PAY_0"]))
pd.DataFrame([corrs], index=["PAY_0"], columns=df.columns)

Корреляция растет к сентябрю.

**Задание 13.**

In [ ]:
import numpy as np

def aggregate_credits():
  credits = {}
  for credit in sorted(set(df["LIMIT_BAL"])):
    clients = df.loc[df["LIMIT_BAL"]==credit]
    without_default = max(sum(clients["default.payment.next.month"]==0), 1)
    ratio = sum(clients["default.payment.next.month"]==1) / without_default
    credits[credit] = {"ratio":round(ratio, 2), "num_clients":clients.shape[0]}
  return credits

credits = aggregate_credits()
for credit in sorted(credits, key=lambda x:credits[x]["ratio"], reverse=True):
  print(str(credit) + " - " + str(credits[credit]))

1. Наибольшая доля дефолтных имеет кредит 327680.0 и 740000.0.
2. Таковых клиентов - трое.
3. Нет.

